In [ ]:
## Modules for Loading Data

import os
import cv2
import numpy as np
import random
import os, numpy, PIL
from PIL import Image
import matplotlib .pyplot as plt
import matplotlib.image as mpimg
from scipy.stats import multivariate_normal
from sklearn.decomposition import PCA
from sklearn import preprocessing

'''
Defining the psuedo code for the EM algorithm for a mixture of gaussian Models
The parameters unknown to us - 
1)the number of models for the given data,
2)the parameters of each of the models
3)the number of iterations

Consider the case where we have to fit 'n' Gaussian Models on a given dataset
The idea is to fit 'n' Gaussian models on a given data

Input: Training data , number of clusters
Output: Estimates of parameters, theta
'''

#Declaring all the global variables
pca = scaler = lold = 0
'''
This function will do all the global operations on the data, 'scaler' and 'pca' should be global variables
'''

def dataOperations(data): # Make sure this RUNS after the data loading is done, so that scaler and pca objects are available
    #Calculating PCA, reducing the dimensionality
    global scaler
    global pca
    scaler = preprocessing.StandardScaler().fit(data)

    Xfeatures = scaler.transform(data)
    #print(Xfeatures)
    #print(Xfeatures.shape)
    #print(Xfeatures)
        
        
    pca = PCA(n_components=50)
    #print(pca.explained_variance_ratio)
    Xnew = pca.fit_transform(data)
    return Xnew # returns the pca of the data

def loadData(flag):
        train_pos_path = '../dataset/train_pos/'
        train_neg_path = '../dataset/train_neg/'
        
        X1 = np.array([np.zeros(10800)]) #This is the feature vector for all positive images
        X0 = np.array([np.zeros(10800)]) #This is the feature vector for all negative images


        #First finding u1, c1 for all the positives
        for filename in os.listdir(train_pos_path):
            imgpath = train_pos_path + filename
            img = cv2.imread(imgpath,-1)
            #print(img.shape)
            x = np.asarray(img)
            x = np.reshape(x,(1,10800))
            #print(x.shape)
            X1 = np.dstack([X1,x]) 
        #print(X1.shape)

        for filename in os.listdir(train_neg_path):
            imgpath = train_neg_path + filename
            img = cv2.imread(imgpath,-1)
            #print(img.shape)
            x = np.asarray(img)
            x = np.reshape(x,(1,10800))
            #print(x.shape)
            X0 = np.dstack([X0,x])
        #print(X0.shape)

        Y1 = np.array([np.ones(1000)])
        #print(Y1.shape)

        Y0 = np.array([np.zeros(1000)])
        #print(Y0.shape)
        
        X1 = np.delete(X1, 0, axis=2)
        X1 = np.reshape(X1,(1000,10800))
        #print(X1.shape)
        X0 = np.delete(X0, 0, axis=2)
        X0 = np.reshape(X0,(1000,10800))
        #print(X0.shape)
        
        if flag == 1:
            return X1
        else:
            return X0

In [2]:
##  Defining the EM module

import os
import cv2
import numpy as np
import random
import os, PIL
from PIL import Image
import matplotlib .pyplot as plt
import matplotlib.image as mpimg
from scipy.stats import multivariate_normal
from sklearn.decomposition import PCA
from sklearn import preprocessing

dictor=0
'''
    Iterations - Max number of iterations the model is going to run till
    Clusters - Max number of clusters
    Data - Will be the feature vector of the dataset (class-specific) -- this should be SCALED
    Parameters - Will specify the number of parameters ( Gaussian - 2)
''' 
class ExpectationMaximisation:   
    
    def __init__(self,m,c,d,n):
        self.maxiters = m
        self.clusters = c
        self.data = d
        self.nparameters = n
    '''
    For defining the EM algorithm
    '''
    def eMaximisation(self):
        global lold;
        cthreshold = 0.1
        iters = 0
        lold = 0
        lnew = L = 1
        
        while((lnew-lold) > cthreshold and iters <= self.maxiters):
            if(iters==0):
                lold = 0
            else:
                lold = L
            iters = iters+1
            x = self.data.shape[0]
            y = self.data.shape[1]

            likelihood = np.zeros((x,self.clusters))
            r = np.zeros((x,self.clusters))
            lnew = np.zeros(self.clusters)
            unew = np.zeros((self.clusters,x))
            cnew = np.zeros((self.clusters,x*x))

            model = np.zeros(self.clusters)

            for i in range(x):
                #every data element is data[i]
                for k in range(self.clusters):
                    #fit the model with the u(k) and sigma(k)
                    model[k] = multivariate_normal(dictor[i][0],dictor[i][1]) # the [0] is mean and the [1] is covariance

                    d = inputTransform(self.data[i,:])
                    likelihood[i,k] = l[k] * model[k].pdf(d)
                #for ends

                for k in range(self.clusters):
                    sumk = np.sum(likelihood,axis=1)[k]
                    r[i,k] = likelihood[i,k]/sumk
                #for ends
            #for ends

            ## The M step starts here
            for k in range(self.clusters):
                lnew[k] = np.sum(r[:,k],axis=0)/np.sum(r) #this is the new prior
                unew[k] = np.sum(r[:,k]*self.data[i,:],axis=0)/np.sum(r[:,k],axis=0) #this is the updated mean
                cnew[k] = np.sum(r[:,k]*(self.data-unew[k]) * np.transpose(self.data-unew[k]),axis=0)/np.sum(r[:,k],axis=0) # this is the updates covariance
            #for ends

            #find the data log likelihood
            for i in range(x):
                for k in range(self.clusters):
                    model[k] = multivariate_normal(dictor[i][0],dictor[i][1]) # the [0] is mean and the [1] is covariance
                    d = inputTransform(self.data[i,:])
                    prob = l[k]*model[k].pdf(d)
                    p = np.sum(prob)
                lg = np.log(p)
            L = np.sum(lg) #finding the log-likelihood
            lnew = L
            
        a = [model,l]
        return a

    '''
    Calculates the transform of the data using pca and scaled
    '''   
    def inputTransform(self,x):
        x = np.reshape(x,(1,10800))
        scaled = scaler.transform(x)
        xtest = pca.transform(scaled)
        return xtest
    '''
    Finding the covariance of the data
    '''     
        
    def findCov(self,data):
        x = data.shape[0]
        y = data.shape[1]
        covariance = np.cov(data.reshape(y,x))
        return covariance
    
    '''
    For initiliasing all the variables which will be used for calculating E- step and M-step
    '''
    def initializationEM(self,data):
        x = data.shape[0]
        y = data.shape[1]
        #print(x)
        #print(y)
        
        l = np.ones(self.clusters)/self.clusters #setting the prior initially to 1/classes
        
        mean= np.zeros((self.clusters,y)) # initilaises the mean to a random row in the dataset
        #np.random.seed(2)
        #idx=np.random.randint(x,size=1)
        
        for i in range(self.clusters):
            idx = np.random.randint(x,size=1)
            mean[i,:] = data[idx,:]
        
        cov = self.findCov(data) #this finds the covariance of the data
        
        #Initialising the parameters
        #The parameters will be mean and covariance
        
        pdict = {}
        for i in range(self.clusters):
            pdict[i] = [mean[i,:],cov]
        return pdict
    '''
    Loads data into feature vectors
    '''        
    
    def main():
        #global ExpectationMaximisation
        global dicto
        #data loading
        data1 = loadData(1)
        print(data1.shape)
        
        #dataoperations
        dataNew = dataOperations(data1)
        print(dataNew.shape)
        
        # Creating EM object with the parameters
        # Header parameters - maxiters,clusters,data,parameters
        em = ExpectationMaximisation(500,2,dataNew,2)
        #print(type(em))
        
        #initialisation
        dictor = em.initializationEM(dataNew)
        #print(dicto)
        
        #while the EM does not converge
        result = em.eMaximisation()
        print("Model and the corresponding parameters",result)

    if __name__ == '__main__':
        main()

(1000, 10800)
(1000, 50)


NameError: global name 'ExpectationMaximisation' is not defined

In [ ]:
print(dicto[0].value[1].shape)